### Create CSVs with all results for each one of the 3 types of models (Simple CNN, Base Model and ResNet-20) 

In [1]:
import pandas as pd

In [27]:
results_cifar10 = pd.read_csv('results_cifar10.csv')
results_cifar10 = results_cifar10[results_cifar10['clf'] == 'lr']

In [28]:
results_cifar100_coarse = pd.read_csv('results_cifar100_coarse.csv')
results_cifar100_fine = pd.read_csv('results_cifar100_fine.csv')
results_fashion_mnist = pd.read_csv('results_fashion_mnist.csv')

In [29]:
all_results = results_cifar10.copy()
all_results = all_results.append(results_cifar100_coarse, ignore_index=True)
all_results = all_results.append(results_cifar100_fine, ignore_index=True)
all_results = all_results.append(results_fashion_mnist, ignore_index=True)

In [30]:
all_results['full_ds_name'] =\
        all_results['dataset'].map(str) + '_' +\
        all_results['color_space'].map(str) + '_' +\
        all_results['n_colors'].map(str)

In [54]:
all_results = all_results[['model', 'full_ds_name', 'test_acc']]

In [96]:
all_results['model'] = all_results['model'].str.replace('_cifar10_',' ')
all_results['model'] = all_results['model'].str.replace('.h5','')

all_results['model'] = all_results['model'].str.replace('simple','Simple CNN')
all_results['model'] = all_results['model'].str.replace('all_conv','Base Model CNN')
all_results['model'] = all_results['model'].str.replace('resnet20','ResNet-20')

all_results['model'] = all_results['model'].str.replace('rgb256','rgb')

In [99]:
base_models = ['Simple CNN', 'Base Model CNN', 'ResNet-20']
ds_names = sorted(all_results['full_ds_name'].unique())

dict_out = {}

for bm in base_models:
    results_base_model = all_results[all_results['model'].str.contains(bm)]
    model_names = sorted(results_base_model['model'].unique())
    
    dict_aux = {}
    for m in model_names:
        results_model = results_base_model[results_base_model['model'].str.contains(m)]
        dict_aux[m] = []
        for ds in ds_names:
            aux = ds
            if ('rgb' in ds) and ('rgb' not in m):
                aux = ds.replace('rgb', 'gray')
            
            dict_aux[m].append(results_model[results_model['full_ds_name'] == aux]['test_acc'].values[0])
            
    df_aux = pd.DataFrame.from_dict(dict_aux)
    df_aux['dataset'] = ds_names
    df_aux = df_aux[['dataset'] + model_names]
    dict_out[bm] = df_aux.copy()

In [101]:
dict_out['Base Model CNN']

,dataset,Base Model CNN gray128,Base Model CNN gray16,Base Model CNN gray256,Base Model CNN gray32,Base Model CNN gray64,Base Model CNN gray8,Base Model CNN rgb
0,cifar100_coarse_gray_128,0.2230,0.2186,0.2110,0.2166,0.2237,0.2132,0.2124
1,cifar100_coarse_gray_16,0.2126,0.2069,0.2059,0.1972,0.2046,0.2112,0.1813
2,cifar100_coarse_gray_256,0.2170,0.2057,0.2061,0.2193,0.2248,0.2043,0.1990
3,cifar100_coarse_gray_32,0.2152,0.2030,0.1928,0.2138,0.2285,0.2052,0.2058
4,cifar100_coarse_gray_64,0.2221,0.1970,0.2115,0.2151,0.2273,0.2052,0.2077
5,cifar100_coarse_gray_8,0.1876,0.1968,0.1622,0.1804,0.1810,0.1919,0.1672
6,cifar100_coarse_rgb_256,0.2170,0.2057,0.2061,0.2193,0.2248,0.2043,0.2174
7,cifar100_fine_gray_128,0.1222,0.1318,0.1195,0.1327,0.1359,0.1325,0.1228
8,cifar100_fine_gray_16,0.1150,0.1247,0.1089,0.1234,0.1209,0.1299,0.1142
9,cifar100_fine_gray_256,0.1267,0.1210,0.1236,0.1319,0.1339,0.1296,0.1238


In [102]:
dict_out.keys()

dict_keys(['Simple CNN', 'Base Model CNN', 'ResNet-20'])

In [103]:
for m in dict_out:
    aux = 'resnet20'
    if m == 'Base Model CNN':
        aux = 'base_model'
    elif m == 'Simple CNN':
        aux = 'simple'
    
    dict_out[m].to_csv('results_%s.csv' % (aux), index=False)